- 保监会 相关性模型 1 预处理

# 基本设置

In [4]:
import jieba
import sys
import re
import time
import string

%matplotlib inline
import numpy as np
import pandas as pd
# import pre_cor
import os

import warnings
warnings.filterwarnings('ignore')

In [5]:
def set_ch():
    '''
    功能：设定绘图时显示中文
    '''	
    from pylab import mpl
    mpl.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体
    mpl.rcParams['axes.unicode_minus'] = False   # 解决保存图像是负号'-'显示为方块的问题
set_ch()

# train_set

## 导入

In [8]:
train_set = pd.read_csv('data/train_set.csv')
# cor_1.columns = ['title', 'sensitivity', 'content']
print(train_set.shape)
train_set.head()

(102277, 4)


,id,article,word_seg,class
0,0,7368 1252069 365865 755561 1044285 129532 1053...,816903 597526 520477 1179558 1033823 758724 63...,14
1,1,581131 165432 7368 957317 1197553 570900 33659...,90540 816903 441039 816903 569138 816903 10343...,3
2,2,7368 87936 40494 490286 856005 641588 145611 1...,816903 1012629 957974 1033823 328210 947200 65...,12
3,3,299237 760651 299237 887082 159592 556634 7489...,563568 1239563 680125 780219 782805 1033823 19...,13
4,4,7368 7368 7368 865510 7368 396966 995243 37685...,816903 816903 816903 139132 816903 312320 1103...,12


## 统计分析

In [10]:
train_set['class'].value_counts().shape

(19,)

## 分割数据

In [11]:
from collections import Counter
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(train_set['word_seg'].tolist(), 
                                                    train_set['class'].tolist(), 
                                                    test_size=0.3, random_state=42)
print('训练集：',len(y_train))
print('训练集-各类数量：',Counter(y_train))
print('测试集：',len(y_test))
print('测试集-各类数量：',Counter(y_test))

训练集： 71593
训练集-各类数量： Counter({3: 5818, 13: 5559, 9: 5344, 15: 5269, 18: 5018, 8: 4894, 6: 4847, 14: 4759, 19: 3879, 12: 3759, 1: 3687, 10: 3482, 4: 2635, 11: 2478, 16: 2200, 17: 2163, 7: 2125, 2: 2011, 5: 1666})
测试集： 30684
测试集-各类数量： Counter({3: 2495, 13: 2348, 9: 2331, 15: 2242, 8: 2078, 18: 2048, 6: 2041, 14: 1981, 1: 1688, 19: 1645, 12: 1567, 10: 1481, 4: 1189, 11: 1093, 16: 1020, 17: 931, 7: 913, 2: 890, 5: 703})


## model

In [16]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2

In [ ]:
pipeline = Pipeline([
    ('tf_idf', Pipeline([
        ('counts', CountVectorizer(max_df=0.95, min_df=2)),
        ('tf_idf', TfidfTransformer()),
        ('chi', SelectKBest(chi2, k=20000))
        ])),
    ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=19))
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

In [ ]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

# test_set

In [9]:
test_set = pd.read_csv('data/test_set.csv')
# cor_1.columns = ['title', 'sensitivity', 'content']
print(test_set.shape)
test_set.head()

20180808/mfa_uncor_english_2.xlsx
(5497, 2)


,title,content
0,Thai vendors rubbish criticism of famous Mae K...,
1,French Politician Arrested for Justifying Terr...,﻿ GOOGL 2.07% The parent company o...
2,The classic explanation for the Black Death pl...,Bob Gamgort has sold Americans Ko...
3,BRIEF-Sinopec Oilfield Service Gets Approval T...,* Bond yields drop as China dismisses repo...
4,BRIEF-Sinopec Oilfield Service Says Lock-Up Pe...,* Bonds give up ground as ECB minutes sugg...


# 保存本文件

In [70]:
if 0:
    import datetime as dt
    
    def output_HTML(read_file, output_file):
        from nbconvert import HTMLExporter
        import codecs
        import nbformat
        exporter = HTMLExporter()
        # read_file is '.ipynb', output_file is '.html'
        output_notebook = nbformat.read(read_file, as_version=4)
        output, resources = exporter.from_notebook_node(output_notebook)
        codecs.open(output_file, 'w', encoding='utf-8').write(output)

    html_file_folder = 'html_files'
    if not os.path.exists(html_file_folder):
        os.makedirs(html_file_folder)

    today = dt.datetime.now().strftime('%Y%m%d')
    current_file = 'circ_cor_model_1_pre.ipynb'
    output_file = 'html_files\%s_%s.html'%(os.path.splitext(current_file)[0], today)
    output_HTML(current_file, output_file)